In [8]:
! pip install python-dotenv
! pip install -qU langchain-openai
! pip install -qU langchain_community beautifulsoup4
! pip install faiss-cpu
! pip install -qU langchain-groq
! pip install langchain_core

In [3]:
env_content = """
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT=https://api.smith.langchain.com
LANGSMITH_API_KEY=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
LANGSMITH_PROJECT=pr-minty-canvas-57
GROQ_API_KEY=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
"""

with open(".env", "w") as f:
    f.write(env_content.strip())

print(".env file created successfully!")

.env file created successfully!


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7deaeaf76010>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7deae9fecdd0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming."),
]
model.invoke(messages)

AIMessage(content="J'adore programmer.  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 26, 'total_tokens': 35, 'completion_time': 0.016363636, 'prompt_time': 0.00014854, 'queue_time': 0.021349689, 'total_time': 0.016512176}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0673043c-2c9c-4441-baa8-ef8b514ed588-0', usage_metadata={'input_tokens': 26, 'output_tokens': 9, 'total_tokens': 35})

In [11]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(model.invoke(messages))

"J'adore programmer. \n"

In [12]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

"J'adore programmer. \n"

In [15]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [17]:
result=prompt.invoke({"language":"French","text":"Hello"})
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [18]:
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [19]:
! pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [20]:
import streamlit as st
from langchain_groq import ChatGroq

# Initialize the model
model = ChatGroq(model="Gemma2-9b-It")

# Streamlit app layout
st.set_page_config(page_title="Gemma Chatbot", layout="centered")
st.title("🤖 Gemma Chatbot")
st.markdown("Chat with the **Gemma2-9b-It** model for translations and more!")

# Initialize session state for chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Display chat history
st.markdown("### Chat History")
for message in st.session_state.chat_history:
    if message["role"] == "human":
        st.markdown(f"**You:** {message['content']}")
    else:
        st.markdown(f"**Gemma:** {message['content']}")

# User input
user_input = st.text_input("Your message:", placeholder="Type your message here...")

if st.button("Send"):  # Handle user input
    if user_input.strip():
        # Add user message to chat history
        st.session_state.chat_history.append({"role": "human", "content": user_input})

        # Prepare model messages
        model_messages = [
            ("system", "You are a helpful assistant. Respond to the user."),
        ] + [(msg["role"], msg["content"]) for msg in st.session_state.chat_history]

        try:
            # Get response from the model
            response = model.invoke(model_messages)
            st.session_state.chat_history.append({"role": "assistant", "content": response})
            st.experimental_rerun()  # Refresh the chat
        except Exception as e:
            st.error(f"An error occurred: {e}")
    else:
        st.warning("Please enter a message.")

# Footer
st.markdown("---")
st.markdown(
    "Developed with ❤️ using [LangChain](https://langchain.com) and [Streamlit](https://streamlit.io)."
)

2025-01-25 15:58:57.918 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-25 15:58:57.941 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-25 15:58:58.532 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-01-25 15:58:58.550 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-25 15:58:58.554 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-25 15:58:58.564 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-25 15:58:58.571 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

DeltaGenerator()